# Deep Learning and Neural Networks 


## Deep learning / Deep neural network

Deep learning is a group of machine learning algorithms that specifically focuses on utilizing neural network architecture.
The neural network approach attempt to mimic the function of biological neurons by stacking layers of neuron units that can pass information.
The network will pass the inforamtion back and forth to adjust the weights of neuron units using backpropagation algorithm based on the objective function (loss function, criterion) that we provide.

The advantages of using deep learning approach includes:
- Modularization
- Universality theorem (one hidden layer is enough, but much effective while deeper)
- End-to-end learning (less feature engineering---if you don't know what features are important at all)

![adv](https://raw.githubusercontent.com/ckbjimmy/2018_mlw/master/img/adv.png)

[Source] Courtesy by Prof. HY Lee (NTU)

However, there are still some cons of using deep neural networks such as interpretability and heavy computation.

As a promising algorithmic approach for predictive analytics (and also the hype of deep learning), we would like to introduce you how to use deep neural networks for clinical machine learning problems.

The mathematical details of neural network approach will not be emphasized in the tutorial.
The foundation of deep neural network is using **gradient descent** to update parameters, and all gradients are computed by **backpropagation algorithm**.
Please refer to [Deep Learning book by Goodfellow et al.](http:/www.deeplearningbook.org/) for detailed information.  

To build the neural network without too much efforts in python, we adopt the high-level python deep learning API, **Keras**, to train neural networks.

After going through this tutorial, we hope that you will understand how to use keras to design and build simple neural network for classification problems, and how to evaluate the keras neural network models.

In this tutorial, we directly use the most complicated data, PhysioNet dataset, as an example to demonstrate the performance of deep learning models.
Please refer to tutorial Part I for all details of data preprocessing.

In [ ]:
! pip install -q -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://raw.githubusercontent.com/ckbjimmy/2018_mlw/master/data/PhysionetChallenge2012_data.csv')
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

Before training neural network, we need to do some transformations for our data.

In [ ]:
X = Imputer(missing_values=np.nan, strategy='mean').fit(X).transform(X)
X = StandardScaler().fit(X).transform(X)

# Encoding Categorical Data
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

Next, we import keras modules and construct a very simple three-layer neural network architecture with input, hidden and output layers.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
from IPython.display import Image
keras.__version__

There are many variants of deep neural networks.

![nn](https://raw.githubusercontent.com/ckbjimmy/2018_mlw/master/img/nn.png)

[Source] Deep Learning, Chapter 1 (Goodfellow, 2016)

We will start from the simplest one, feedforward neural network, which is similar to the neural network architecture shown in the following figure.

![nn](https://cdn-images-1.medium.com/max/1600/1*QVIyc5HnGDWTNX3m-nIm9w.png)

[Source] https://medium.com/@curiousily/tensorflow-for-hackers-part-iv-neural-network-from-scratch-1a4f504dfa8

In our first neural network model, we will construct the one that

- take the input and pass them into the 64-dimension first hidden layer, 
- take the output of the first layer and pass them into the  8-dimension second layer,
- take the output of the second layer and pass them into the last layer for prediction,
- the output of the last layer is the prediction.

One more hidden layer than the above figure.

In keras, we use `Sequential()` as the skeleton of the neural network model, and sequentially add the layer on it.
After building the layers, we need to compile the model and defined the optimizer, loss function and evaluation metrics to optimize our model.
In this example, we use the optimizer called `adam`, to minimize the value of loss function `binary_crossentropy` (if you work on the regression problem, remember to change to `mse`), and judge by accuracy.

In [ ]:
# initialize neural network
clf = Sequential()

# first hidden layer for input data
clf.add(Dense(units=64, 
              kernel_initializer='uniform',
              activation='relu', 
              input_dim=X.shape[1]))

# second hidden layer
clf.add(Dense(units=8, 
              kernel_initializer='uniform', 
              activation='relu'))

# the last hidden layer for output
clf.add(Dense(units=1, 
              kernel_initializer='uniform', 
              activation='sigmoid'))

# compile the network
clf.compile(optimizer='adam', 
            loss='binary_crossentropy', 
            metrics=['accuracy'])


print(clf.summary())

Then, it's time to train the model!
Smaller batch size usually yields better performance, yet with slow speed.
The epoch size of $n$ means that the algorithm will go through whole dataset $n$ times.

After training, we visualize the training history and see how the accuracy and loss went during the process.
We also use confusion matrix and **pycm** for computing the performance.


In [ ]:
# fit model
history = clf.fit(x=X_train, y=y_train,
                    validation_split=0.2,  
                    batch_size=100,
                    epochs=20, 
                    verbose=1)

In [ ]:
import matplotlib.pyplot as plt  
def show_train_history(train_history, train, validation):  
    plt.plot(train_history.history[train])   
    plt.plot(train_history.history[validation])  
    plt.title('Train History')  
    plt.ylabel(train)  
    plt.xlabel('Epoch')  
    plt.legend(['train', 'validation'], loc='upper left')  
    plt.show() 
    
show_train_history(history, 'accuracy', 'val_accuracy')
show_train_history(history, 'loss', 'val_loss')

In [ ]:
# we use the trained model to predict the label of test set
y_pred = clf.predict(X_test)
y_pred = [1 if i[0] > 0.5 else 0 for i in y_pred]

# show the performance using confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred) 
print(cm)

In [ ]:
# we use pycm to show the performance

from pycm import ConfusionMatrix

cm = ConfusionMatrix(actual_vector=y_test, predict_vector=y_pred)
print(cm)

After seeing a simple example, we discuss more about the network.
Normally, when building a neural network, we will do the following steps:

### Define a set of function (Network architecture)
We can see that the neuron unit can be regarded as a logistic regression-like operation.
In general, the unit will do `activation_function(weight * input + bias)` computation.

![ff](https://raw.githubusercontent.com/ckbjimmy/2018_mlw/master/img/ff.png)

[Source] Courtesy by Prof. HY Lee (NTU)

In the feedforward model, the layer function we used is `Dense` (fully-connected layer).
For the `Dense` layer, you need to assign the input size, output size (`units`), and the activation function.
For activation function, you can try the following options: `softplus`, `softsign`, `relu`, `tanh`, `sigmoid`, `hard_sigmoid`, `linear`, depends on your needs.
However, `relu` are the most commonly used functions for intermediate layers, and `sigmoid`/`softmax` for the last layer (binary/multiclass).

Multiple hidden layers are good, but there is no theorerical consensus how many layers / how many neurons / what hyperparameters you need to use to get the best performance. (That's why some people think that deep learning approach has become [alchemy](https://www.reddit.com/r/compsci/comments/7jiipp/lecun_vs_rahimi_has_machine_learning_become/))

![stack](https://1.bp.blogspot.com/-gZ7UBZ-wiBE/WZU12kj6wDI/AAAAAAAAKNQ/U8998PG8U30fgzZUWVtQhzl5fpVlCxnSwCLcBGAs/s1600/layers.jpg))

[Source] http://www.cl.cam.ac.uk/~pv273/slides/LOSSlides.pdf

### Goodness of function (Define loss)
The goal of prediction is to minimize the difference between the predicted value and the ground truth label. 
We need to decide how to compute the difference between them, we call this "objective" in deep learning.

![loss](https://raw.githubusercontent.com/ckbjimmy/2018_mlw/master/img/loss.png)

[Source] Courtesy by Prof. HY Lee (NTU)

In the first case, we will use `binary_crossentropy` for the binary classification task. 
For more objectives, please refer to the [keras document](https://keras.io/objectives/).

### Choose the best function (Optimization and parameter setting)
We also need to decide which optimizer will be used in the network. 
The commonly used optimizers include: `SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam`.
The backpropagation algorithm will use the selected optimizer to minimize the objective (loss) function we defined previously.
Finally, we need to decide the parameters `batch_size` and `epochs` for training. 
Batch size influences the speed and performance so you have to tune it and do some experiments.
If you have $1000$ training samples and set `batch_size=50`, then in one epoch (going through all training samples once) you need to update parameters for $\frac{1000}{50}=20$ times, until all "mini-batches" are picked.

For prediction, you can simply use `model.evaluate()` to get the performance metrics, and use `model.predict()` to obtain the predicted results.

### (optional) Exercise of regularization
Regularization is a pratical way to prevent from overfitting problem. 
You may try to add `BatchNormalization` (batch normalization) or `Dropout` (dropout) layers and see whether they will be helpful for modeling.




## Exercise

Now try to play with the breast cancer dataset and see whether deep neural network works well on the smaller dataset.

In [ ]:
from sklearn import datasets

df_bc = datasets.load_breast_cancer()
X = df_bc['data']
y = df_bc['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

clf = Sequential()
clf.add(Dense(units=16, 
              kernel_initializer='uniform',
              activation='relu', 
              input_dim=X.shape[1]))
clf.add(Dense(units=16, 
              kernel_initializer='uniform', 
              activation='relu'))
clf.add(Dense(units=1, 
              kernel_initializer='uniform', 
              activation='sigmoid'))
clf.compile(optimizer='adam', 
            loss='binary_crossentropy', 
            metrics=['accuracy'])
print(clf.summary())

history = clf.fit(x=X_train, y=y_train,
                    validation_split=0.2,  
                    batch_size=100,
                    epochs=50, 
                    verbose=1)

show_train_history(history, 'accuracy', 'val_accuracy')
show_train_history(history, 'loss', 'val_loss')

y_pred = clf.predict(X_test)
y_pred = [1 if i[0] > 0.5 else 0 for i in y_pred]

cm = ConfusionMatrix(actual_vector=y_test, predict_vector=y_pred)
print(cm)

## Convolutional neural network (CNN) for image data

Sometimes we will use image instead of structured tabular data for clinical data predictive analytics.
Here we show a very simple image classification workflow using [MNIST handwriting database](https://en.wikipedia.org/wiki/MNIST_database) since there is no small but suitable medical image dataset for simple demonstration with deep neural network.

Although not business focused, MNIST is already included in the `keras` and we can simply load it through the `keras` provided functions.
But we still need to do reshape the data to the proper format, and normalize them before feed into the model.

First, we will just simply use the feedforward neural network for image classification (10 classes).

In [ ]:
import numpy as np
import pandas as pd
from keras.utils import np_utils
from keras.datasets import mnist

# load MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape the data
X_train = X_train.reshape(60000, 28*28).astype('float32')
X_test = X_test.reshape(10000, 28*28).astype('float32')
  
# normalize the data
X_train = X_train/255
X_test = X_test/255

# for keras classification, we need to use `to_categorical` to transform the label to appropriate format
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
clf = Sequential()
clf.add(Dense(units=256, 
                input_dim=784, 
                kernel_initializer='normal', 
                activation='relu'))
clf.add(Dense(units=256, 
                input_dim=64, 
                kernel_initializer='normal', 
                activation='relu'))
clf.add(Dense(units=10, 
                kernel_initializer='normal', 
                activation='softmax'))
print(clf.summary())



In [ ]:
clf.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

history = clf.fit(x=X_train, y=y_train, 
                    validation_split=0.2,  
                    batch_size=200, 
                    epochs=10,
                    verbose=2)

show_train_history(history, 'accuracy', 'val_accuracy')
show_train_history(history, 'loss', 'val_loss')

scores = clf.evaluate(X_test, y_test)  
print("Accuracy of testing data = {:2.1f}%".format(scores[1]*100.0))

We can see that feedforward neural network can do a very good job on image classification. 
However, we may also use **convolutional neural network** (CNN), to do classification considering spatial information.

The key steps in CNN are:

- Convolution
- Pooling

The idea of convolution is to have **sparse connectivity** (each neuron only connects to parts of the previous layer) and **parameter sharing** (neurons can use the same set of parameters). 

![conv](https://raw.githubusercontent.com/ckbjimmy/2018_mlw/master/img/conv.png)

[Source] Deep Learning, Chapter 9 (Goodfellow, 2016)

The pooling strategy also group the neurons corresponding to the same filter with nearby receptive fields.

![pool](https://raw.githubusercontent.com/ckbjimmy/2018_mlw/master/img/pool.png)

[Source] Deep Learning, Chapter 9 (Goodfellow, 2016)

These characteristics preserve the spatial information, as well as reduce the model size significantly.

The following figure demonstrates the process of CNN on MNIST data.

![cnn](https://codetolight.files.wordpress.com/2017/11/network.png)

[Source] https://codetolight.files.wordpress.com/2017/11/network.png

In the following codes, we will use `keras` to build the same model and see how the performance goes using CNN on MNIST.
You may compare the value of the figure and those in the sequential model below.

We can see that the performance significantly increase using CNN approach for image data.

You can also play with changing the value of parameters such as `filters`,  `activation`,  `pool_size`,  `drop_out`, `Dense` and see how the performance will change.

In [ ]:
from keras.layers import Dropout, Flatten, Conv2D, MaxPool2D 

(X_train, y_train), (X_test, y_test) = mnist.load_data()  
  
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')  
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

X_train = X_train / 255  
X_test = X_test /255  
  
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

clf = Sequential()  
clf.add(Conv2D(filters=10,  
                 kernel_size=(5, 5),  
                 padding='same',  
                 input_shape=(28, 28, 1),  
                 activation='relu',
                 name='conv2d_1')) 
clf.add(MaxPool2D(pool_size=(2,2), 
                    name='max_pooling2d_1'))  
clf.add(Conv2D(filters=20,  
                 kernel_size=(5, 5),  
                 padding='same',  
                 input_shape=(28, 28, 1),  
                 activation='relu',
                 name='conv2d_2'))  
clf.add(MaxPool2D(pool_size=(2,2), 
                    name='max_pooling2d_2'))  
clf.add(Dropout(0.25, name='dropout_1'))
clf.add(Flatten(name='flatten_1'))
clf.add(Dense(100, activation='relu', name='dense_1'))  
#clf.add(Dropout(0.5, name='dropout_2'))
clf.add(Dense(10, activation='softmax', name='dense_2'))
print(clf.summary())



In [ ]:
clf.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])  
  
history = clf.fit(x=X_train, y=y_train,
                    validation_split=0.2,  
                    batch_size=200,
                    epochs=10, 
                    verbose=1)

show_train_history(history, 'accuracy', 'val_accuracy')
show_train_history(history, 'loss', 'val_loss')

scores = clf.evaluate(X_test, y_test)  
print("Accuracy of testing data = {:2.1f}%".format(scores[1]*100.0))

## Recurrent neural network (RNN) for sequential data / text

Text data are even much important for clinical data predictive analytics since they usually include a lot of information that are not saved in the tabular data.
However, preprocessing text data is quite messy and time-consuming (but extremely important).
We won't cover this part in the tutorial but only quickly go through the modeling.

We again show a simple text sentiment classification workflow using [IMDB Movie reviews sentiment classification database](https://keras.io/datasets/), which is already preprocessed and labeled.

For your own text data, please make sure that they are in the same format as the IMDB dataset in `keras`.

For text data, you will use a type of neural network, **recurrent neural network** (RNN) for the sequential information. 
RNN and its variants can be used for different kinds of sequential data such as text, speech, audio or time-series data.

RNN has the feature of memorizing the previous input while training through passing the hidden state to the neural network again and again---which is suitable for sequential modeling.

![rnn](https://raw.githubusercontent.com/ckbjimmy/2018_mlw/master/img/rnn.png)

[Source] Deep Learning, Chapter 10 (Goodfellow, 2016)

Here we just demonstrate two approaches of modeling the IMDB data:

- using a RNN variant, long short term memory neural network (LSTM) for modeling,
- combining CNN and LSTM for modeling (CNN for compressing the sequence information).

For the detailed differences of neural network architecture, please check the figure of neural networks below.

To make the modeling process simpler, we make all sequences (texts) the same length using zero padding.

In [ ]:
from keras.datasets import imdb
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

top_words = 3000
max_review_length = 200
embedding_vecor_length = 32

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# LSTM
clf = Sequential()
clf.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
clf.add(LSTM(100))
clf.add(Dense(1, activation='sigmoid'))
clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(clf.summary())



In [ ]:
history = clf.fit(x=X_train, y=y_train,
                    validation_split=0.2,  
                    batch_size=200,
                    epochs=10, 
                    verbose=1)

show_train_history(history, 'accuracy', 'val_accuracy')
show_train_history(history, 'loss', 'val_loss')

scores = clf.evaluate(X_test, y_test)  
print("Accuracy of testing data = {:2.1f}%".format(scores[1]*100.0))

Finally, we would like to combine CNN and RNN for sequential modeling.
We utilize CNN to reduce the length of sequence and the number of parameters, in order to make the RNN training much efficient and robust (less issue of long-term dependency).

The basic idea is shown in the following figure.

![1dcnn](https://raw.githubusercontent.com/ckbjimmy/2018_mlw/master/img/1dcnn.png)

[Source] Kim, 2014


In [ ]:
from keras.layers.convolutional import Conv1D, MaxPooling1D

# CNN + LSTM
clf = Sequential()
clf.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
clf.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
clf.add(MaxPooling1D(pool_size=2))
clf.add(Dropout(0.2))
clf.add(LSTM(100))
clf.add(Dropout(0.2))
clf.add(Dense(1, activation='sigmoid'))
clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(clf.summary())


In [ ]:
history = clf.fit(x=X_train, y=y_train,
                    validation_split=0.2,  
                    batch_size=200,
                    epochs=10, 
                    verbose=1)

show_train_history(history, 'accuracy', 'val_accuracy')
show_train_history(history, 'loss', 'val_loss')

scores = clf.evaluate(X_test, y_test)  
print("Accuracy of testing data = {:2.1f}%".format(scores[1]*100.0))

Except for the CNN, RNN and LSTM layers we used in the tutorial, you may also want to play with other RNN variants such as gated recurrent units (GRU), bidirectional RNN/LSTM/GRU.
They are all available in Keras and all you need is computational resource.

Deep neural network has many variants and many hyperparameters for fine-tuning.
We are not going to explain all functions above.
If you are interested in the details of the models and want to fine-tune them, please definitely go to [Keras document](https://keras.io/) for more information.

We hope that this tutorial helps you understand the basic of deep neural network---feedforward neural network, CNN and RNN, and how to choose and use them in your clinical machine learning problems.

## Further reading

### Theory and mathematics
- [Deep learning book](http://www.deeplearningbook.org/)
- [Stanford CS224n: Natural Language Processing with Deep Learning](http://web.stanford.edu/class/cs224n/)
- [Stanford CS231n: Convolutional Neural Networks for Visual Recognition](http://cs231n.stanford.edu/)

### Practical
You may want to read the documents of TensorFlow, PyTorch, Keras, Scikit-learn, ... etc. to know how to use them.

- [Tensorflow Programmer's Guide](https://www.tensorflow.org/programmers_guide/)
- [PyTorch Tutorials](https://pytorch.org/tutorials/)
- [Coursera Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning)
- [Google Machine Learning Crash Course
](https://developers.google.com/machine-learning/crash-course/ml-intro)
- [Coursera Machine Learning with TensorFlow on Google Cloud Platform Specialization](https://www.coursera.org/specializations/machine-learning-tensorflow-gcp)